### Intro

This is an ipython notebook that is runnable from the cloud.   Each cell in the notbook can be run seperately by clicking on it and hitting ctrl+enter.   Once a cell is run, all the global variables are saved, so you can run other cells using variable you've already initialized.

First thing you should do is run all the cell by either hitting ctrl+enter for each cell or by choosing **Kernel** and **Restart and run all** from the menu above.

In [141]:
import numpy as np
import random
import matplotlib.pyplot as plt

### Random model code

This section contains functions to generate random monotic models.

In [142]:
def monotonic_pairs(max_dist, seed = None):
    # This function will generate a list of pairs
    # List in form of (x,y), where y is monotonicly decreasing.
    # 0<=x<=1.   0<y.   List includes (1,0) and (0,1).
    # The function is assumed to be piecewise linear.
    # max_dist >= xi-x_(i-1)
    if seed is not None:
        random.seed(seed)
    return random_seq(max_dist, 0,1, 0,1)
        

def random_point(x_min, x_max, y_min, y_max):
    return (random.uniform(x_min, x_max), random.uniform(y_min, y_max))

def random_seq(max_dist, x_min, x_max, y_min, y_max):
    x_mid, y_mid= random_point(x_min, x_max, y_min, y_max)
    if x_max-x_min < max_dist:
        return [(x_mid, y_mid)]
    else:
        return random_seq(max_dist, x_min, x_mid, y_mid, y_max) + \
                    [(x_mid, y_mid)] + \
                    random_seq(max_dist, x_mid, x_max, y_min, y_mid)


def plot_pairs(pairs):
    x,y = zip(*pairs)
    plt.plot(x,y)

def get_monotoic_vals(shells):
    # This function will generate a random sequence with monotonic pairs,
    # then sample it to have exactly the number of points requested.  Linear interpolation.
    # @param num:  Number of points.   
    #
    # Note that the last point will always be (1,0)
    
    # Make sure we have enough points.  

    # TODO:   This is a hack
    # Can do better to make sure each point is separate.   Not sure it makes a difference.
    num = len(shells)
    pairs = monotonic_pairs(0.5/num)
    
    x,y = zip(*pairs)
    return np.interp(shells,x,y)

In [144]:
# Uncomment below lines to see examples:

#for _ in range(10):
#    plot_pairs(monotonic_pairs(0.01))
#    plt.figure()

### Normalization code

These two functions compute the amount of mass/moment per shell.   Right now, we only support a fixed density.

$ mass = 2\pi\rho\int_{r_1}^{r_2}r^2dr = \dfrac{2}{3}\pi\Big[r_2^3-r_1^3\Big]\rho$

$ moment = \dfrac{2}{3}MR^2 = \dfrac{4}{3}\pi\rho\int_{r_1}^{r_2}r^4dr = \dfrac{4}{15}\pi\Big[r_2^5-r_1^5\Big]\rho$+

Thus, given a series of fixed shells with radius $r_i$, there is a series of fixed coefficients $M_i$ and $I_i$ such that:

$ mass = \vec{M}\vec{\rho} $

$ moment =  \vec{I}\vec{\rho} $


**Note:** I believe the fixed coefficients also hold with linear density (though the general equations to compute the mass/moment might be a bit more involved).

The advantage of this method is that if we have two solutions $\vec{\rho_1}$ and $\vec{\rho_2}$, any linear combination of the two solution will also have the same mass/moment.   This allows us to fix yet another variable (e.g. J2) with binary search.

In [167]:
def get_mass_coefficient(rad1, rad2, fixed_density=True):
    if not fixed_density:
        raise NotImplemented("Linear density not implmeneted yet.")
    return 2.0/3.0 * np.pi * (np.power(rad2,3.0) - np.power(rad1, 3.0))
        
        
def get_moment_coefficient(rad1, rad2, fixed_density=True):
    if not fixed_density:
        raise NotImplemented("Linear density not implemented yet.")
    return 4.0/15.0 * np.pi * (np.power(rad2,5.0) - np.power(rad1, 5.0))


def compute_mass(model, fixed_density=True):
    # @param model: This is two arrays, first with outer radii, second with densities
    radii, densities = model
    outer = radii
    inner= np.append([0], radii)
    ranges = zip(inner, outer)
        
    coefficients = [get_mass_coefficient(rad1, rad2, fixed_density) 
                    for (rad1, rad2) in ranges]
        
    return np.array(coefficients).dot(densities)

def compute_moment(model, fixed_density=True):
    # @param model: This is two arrays, first with outer radii, second with densities
    radii, densities = model
    outer = radii
    inner= np.append([0], radii)
    ranges = zip(inner, outer)
        
    coefficients = [get_moment_coefficient(rad1, rad2, fixed_density) 
                    for (rad1, rad2) in ranges]
        
    return np.array(coefficients).dot(densities)

In [168]:
class McPlanet(object):
    def __init__(self, mass, moment, radius, shells=None, num_shells = 100, fixed_density=True):
        # TODO:   Need to think about/ document units.
        self._mass = mass
        self._moment = moment
        self._use_fixed = fixed_density
        self._radius = radius
    
        # Can pass in units as increasing list of radii.
        # Shells are the outer radius of the shells.   Assume shell are touching.
        # num_shells ignored in this case.
        if shells is None:
            self._shells = self._create_radii(num_shells)
        else:
            self._shells = np.array(shells)
            
        self._num_shells = len(self._shells)
    
    def _create_radii(self, num_shells):
        # For now, just equal radii for each shell.
        return (np.array(range(num_shells))+1)/float(num_shells)*self._radius
       
    def _normalize_densities(self, model_points):
        # Assume that our current model is (x,y)
        # Where x is the outer radius (first is zero)
        # And y is the density.   
        #
        # We need to normalize so we match the total mass
        outer = self._shells
        inner= np.append([0], self._shells)
        ranges = zip(inner, outer)
        
        coefficients = [get_mass_coefficient(rad1, rad2, self._use_fixed) 
                        for (rad1, rad2) in ranges]
        
        mass = np.array(coefficients).dot(model_points)
        return model_points*self._mass/mass
        
        
    def create_mass_model(self, seed=None):
        # first generate random monotonic-path
        model = get_monotoic_vals(self._shells/self._radius)
        return self._shells, self._normalize_densities(model)
                 
        

In [169]:
aa= McPlanet(mass = 1000000000, moment= 10, radius =1000, num_shells=100)
model = aa.create_mass_model()
print(model)

(array([  10.,   20.,   30.,   40.,   50.,   60.,   70.,   80.,   90.,
        100.,  110.,  120.,  130.,  140.,  150.,  160.,  170.,  180.,
        190.,  200.,  210.,  220.,  230.,  240.,  250.,  260.,  270.,
        280.,  290.,  300.,  310.,  320.,  330.,  340.,  350.,  360.,
        370.,  380.,  390.,  400.,  410.,  420.,  430.,  440.,  450.,
        460.,  470.,  480.,  490.,  500.,  510.,  520.,  530.,  540.,
        550.,  560.,  570.,  580.,  590.,  600.,  610.,  620.,  630.,
        640.,  650.,  660.,  670.,  680.,  690.,  700.,  710.,  720.,
        730.,  740.,  750.,  760.,  770.,  780.,  790.,  800.,  810.,
        820.,  830.,  840.,  850.,  860.,  870.,  880.,  890.,  900.,
        910.,  920.,  930.,  940.,  950.,  960.,  970.,  980.,  990.,
       1000.]), array([1.22981186e+00, 1.22978257e+00, 1.22957536e+00, 1.22887671e+00,
       1.22882892e+00, 1.22880560e+00, 1.22880462e+00, 1.22879016e+00,
       1.22748075e+00, 1.22693336e+00, 1.22296812e+00, 1.22152003e+00,


In [170]:
compute_mass(model)

1000000000.0

In [171]:
compute_moment(model)

266148235226334.62